In [1]:
from pprint import pprint
import json
import pandas as pd

In [2]:
with open('player_dict-college-withCareerwithYear.json') as json_file:
    player_dict = json.load(json_file)

In [3]:
def get_tablehead(targetHeader:str) -> list:
    '''
    Function that takes input of the desired table header.  Returns a list of headers for the desired table.
    '''

    for letter in player_dict:
        for i in range(len(player_dict[letter]['playerIds'])):
            if player_dict[letter]['Body'][i]['proLink'] != None:
                
                try:
                    header = [f for f in player_dict[letter]['Body'][i]["collegeData"].keys()][1]
                except IndexError:
                    continue 

                if header == targetHeader:
                    if targetHeader == 'all_passing':
                        return ["playerId", "year"] + list(player_dict[letter]['Body'][i]["collegeData"][targetHeader]['career'].keys()) + \
                    list(player_dict[letter]['Body'][i]["collegeData"]["all_rushing"]['career'].keys())
                    else:
                        return ["playerId","year"] + list(player_dict[letter]['Body'][i]["collegeData"][targetHeader]['career'].keys())
                                 

In [4]:
def create_table(statgroup:str):
    '''
    Function that will take input of desired table name and create a nested array with the table headers in index zero
    prepping the object to be converted into a Pandas DF.
    Dependency on function get_tablehead
    '''
    
    table= []
    table.append(get_tablehead(statgroup))
    for letter in player_dict:

        print("Starting at letter", letter)

        for i in range(len(player_dict[letter]['playerIds'])):

            if 'collegeData' in player_dict[letter]['Body'][i].keys(): 
                dataheaders = [f for f in player_dict[letter]['Body'][i]["collegeData"].keys()]

                try:
                    targetHeader = dataheaders[1]
                except IndexError:
                    print(f"No college stats for {player_dict[letter]['playerIds'][i]}")
                    continue  

                if targetHeader == statgroup:
                    #position = player_dict[letter]['Body'][i]["collegeData"]["draftData"]["Position"]
                    years = [g for g in player_dict[letter]['Body'][i]["collegeData"][dataheaders[1]] if g != 'career']

                    for year in years:
                        temp_list = []
                        temp_list.append(player_dict[letter]['playerIds'][i])
                        #temp_list.append(position)
                        temp_list.append(year)
                        datapoints = list(player_dict[letter]['Body'][i]["collegeData"][targetHeader][year].keys())
                        for datapoint in datapoints:
                            temp_list.append(player_dict[letter]['Body'][i]["collegeData"][targetHeader][year][datapoint])
                        if statgroup == 'all_passing':
                            try:
                                rushingData = list(player_dict[letter]['Body'][i]["collegeData"]["all_rushing"][year].keys())
                                for rushPoint in rushingData:
                                    temp_list.append(player_dict[letter]['Body'][i]["collegeData"]["all_rushing"][year][rushPoint])
                            except KeyError:
                                print(f"No rushing for {player_dict[letter]['playerIds'][i]}")
                                for ix in range(17):
                                    temp_list.append("")
                            
                        table.append(temp_list)

    return table


In [5]:
'''
Test table for quarterbacks.
'''

test_table = create_table('all_passing')
print(len(test_table))

Starting at letter a
No college stats for george-achica-1
No college stats for paul-adams-3
No college stats for alex-agase-1
No college stats for siaosi-aiono-1
No college stats for frankie-albert-1
No college stats for trev-alberts-1
No college stats for austin-albrecht-1
No college stats for ki-aldrich-1
No college stats for vadal-alexander-1
No college stats for kofi-amichia-1
No college stats for dick-anderson-2
No college stats for eddie-anderson-1
No college stats for mike-anderson-8
No college stats for pete-anderson-1
No college stats for scott-appleton-1
No college stats for kareem-are-1
No college stats for bill-atessis-1
No college stats for travis-averill-1
Starting at letter b
No college stats for david-baas-1
No college stats for bob-babich-1
No college stats for ed-bagdon-1
No college stats for frank-baker-1
No college stats for sam-baker-1
No college stats for david-bakhtiari-1
No college stats for burr-baldwin-1
No college stats for dave-ball-1
No college stats for jo

In [6]:
'''
Validating that table headers are in index 0 and data is in the following nested arrays.
'''

pprint(test_table[:10])

[['playerId',
  'year',
  'school_name',
  'conf_abbr',
  'class',
  'pos',
  'g',
  'pass_cmp',
  'pass_att',
  'pass_cmp_pct',
  'pass_yds',
  'pass_yds_per_att',
  'adj_pass_yds_per_att',
  'pass_td',
  'pass_int',
  'pass_rating',
  'school_name',
  'conf_abbr',
  'class',
  'pos',
  'g',
  'rush_att',
  'rush_yds',
  'rush_yds_per_att',
  'rush_td',
  'rec',
  'rec_yds',
  'rec_yds_per_rec',
  'rec_td',
  'scrim_att',
  'scrim_yds',
  'scrim_yds_per_att',
  'scrim_td'],
 ['sam-adkins-1',
  '1974',
  'Wichita State',
  'MVC',
  '',
  'QB',
  '11',
  '123',
  '240',
  '51.3',
  '1404',
  '5.9',
  '4.3',
  '4',
  '10',
  '97.6',
  'Wichita State',
  'MVC',
  '',
  'QB',
  '11',
  '129',
  '-37',
  '-0.3',
  '1',
  '',
  '',
  '',
  '',
  '129',
  '-37',
  '-0.3',
  '1'],
 ['sam-adkins-1',
  '1975',
  'Wichita State',
  'MVC',
  '',
  'QB',
  '11',
  '105',
  '207',
  '50.7',
  '1217',
  '5.9',
  '5.5',
  '3',
  '3',
  '102.0',
  'Wichita State',
  'MVC',
  '',
  'QB',
  '11',
  '107'

In [8]:
def create_csv(statGroup:str):
    '''
    Main function that will take the input of the desired stat group, then create a normalized table and save it to a csv file.
    Not entering a valid value of "all_passing", "all_rushing", "all_receiving", or "all_defense" will result in invalid input message.
    Dependency on functions get_tablehead, create_table
    '''
    
    if statGroup not in {"all_passing", "all_rushing", "all_receiving", "all_defense"}:
        print("Invalid stat input.  Use one of the following: all_passing, all_rushing, all_receiving, or all_defense")
        return
    
    table = create_table(statGroup)
    df = pd.DataFrame(table[1:], columns = table[0])
    df['year'] = df['year'].str.replace('\W', '')
    df.to_csv(f"{statGroup}.csv",index=False)
    return

In [9]:
create_csv("all_passing")

Starting at letter a
No college stats for george-achica-1
No college stats for paul-adams-3
No college stats for alex-agase-1
No college stats for siaosi-aiono-1
No college stats for frankie-albert-1
No college stats for trev-alberts-1
No college stats for austin-albrecht-1
No college stats for ki-aldrich-1
No college stats for vadal-alexander-1
No college stats for kofi-amichia-1
No college stats for dick-anderson-2
No college stats for eddie-anderson-1
No college stats for mike-anderson-8
No college stats for pete-anderson-1
No college stats for scott-appleton-1
No college stats for kareem-are-1
No college stats for bill-atessis-1
No college stats for travis-averill-1
Starting at letter b
No college stats for david-baas-1
No college stats for bob-babich-1
No college stats for ed-bagdon-1
No college stats for frank-baker-1
No college stats for sam-baker-1
No college stats for david-bakhtiari-1
No college stats for burr-baldwin-1
No college stats for dave-ball-1
No college stats for jo

In [10]:
create_csv("all_rushing")

Starting at letter a
No college stats for george-achica-1
No college stats for paul-adams-3
No college stats for alex-agase-1
No college stats for siaosi-aiono-1
No college stats for frankie-albert-1
No college stats for trev-alberts-1
No college stats for austin-albrecht-1
No college stats for ki-aldrich-1
No college stats for vadal-alexander-1
No college stats for kofi-amichia-1
No college stats for dick-anderson-2
No college stats for eddie-anderson-1
No college stats for mike-anderson-8
No college stats for pete-anderson-1
No college stats for scott-appleton-1
No college stats for kareem-are-1
No college stats for bill-atessis-1
No college stats for travis-averill-1
Starting at letter b
No college stats for david-baas-1
No college stats for bob-babich-1
No college stats for ed-bagdon-1
No college stats for frank-baker-1
No college stats for sam-baker-1
No college stats for david-bakhtiari-1
No college stats for burr-baldwin-1
No college stats for dave-ball-1
No college stats for jo

In [11]:
create_csv("all_receiving")

Starting at letter a
No college stats for george-achica-1
No college stats for paul-adams-3
No college stats for alex-agase-1
No college stats for siaosi-aiono-1
No college stats for frankie-albert-1
No college stats for trev-alberts-1
No college stats for austin-albrecht-1
No college stats for ki-aldrich-1
No college stats for vadal-alexander-1
No college stats for kofi-amichia-1
No college stats for dick-anderson-2
No college stats for eddie-anderson-1
No college stats for mike-anderson-8
No college stats for pete-anderson-1
No college stats for scott-appleton-1
No college stats for kareem-are-1
No college stats for bill-atessis-1
No college stats for travis-averill-1
Starting at letter b
No college stats for david-baas-1
No college stats for bob-babich-1
No college stats for ed-bagdon-1
No college stats for frank-baker-1
No college stats for sam-baker-1
No college stats for david-bakhtiari-1
No college stats for burr-baldwin-1
No college stats for dave-ball-1
No college stats for jo

In [12]:
create_csv("all_defense")

Starting at letter a
No college stats for george-achica-1
No college stats for paul-adams-3
No college stats for alex-agase-1
No college stats for siaosi-aiono-1
No college stats for frankie-albert-1
No college stats for trev-alberts-1
No college stats for austin-albrecht-1
No college stats for ki-aldrich-1
No college stats for vadal-alexander-1
No college stats for kofi-amichia-1
No college stats for dick-anderson-2
No college stats for eddie-anderson-1
No college stats for mike-anderson-8
No college stats for pete-anderson-1
No college stats for scott-appleton-1
No college stats for kareem-are-1
No college stats for bill-atessis-1
No college stats for travis-averill-1
Starting at letter b
No college stats for david-baas-1
No college stats for bob-babich-1
No college stats for ed-bagdon-1
No college stats for frank-baker-1
No college stats for sam-baker-1
No college stats for david-bakhtiari-1
No college stats for burr-baldwin-1
No college stats for dave-ball-1
No college stats for jo

## Get draft info

## Create normalized table of just draft data.  One row per player.

In [156]:
player_dict["a"].keys()

dict_keys(['playerIds', 'Body'])

In [14]:
'''
Itterate through player_dict object and crate a nested array of dictionaries containing draft information for drafted players.
'''

draftedBy = []

for letter in player_dict:
        print("Starting at letter", letter)
        for i in range(len(player_dict[letter]['playerIds'])):
            if 'collegeData' in player_dict[letter]['Body'][i].keys() and "draftedBy" in player_dict[letter]['Body'][i]["collegeData"]["draftData"]:
                temp = {}
                temp["playerId"] = player_dict[letter]['playerIds'][i]
                temp["draftedPos"] = player_dict[letter]['Body'][i]["collegeData"]["draftData"]["Position"]
                temp["draftedBy"] = player_dict[letter]['Body'][i]["collegeData"]["draftData"]["draftedBy"]
                temp["draftedOverall"] = player_dict[letter]['Body'][i]["collegeData"]["draftData"]["overallPick"]
                temp["round"] = player_dict[letter]['Body'][i]["collegeData"]["draftData"]["round"]
                temp["draftedYear"] = player_dict[letter]['Body'][i]["collegeData"]["draftData"]["draftedYear"]
                draftedBy.append(temp)


Starting at letter a
Starting at letter b
Starting at letter c
Starting at letter d
Starting at letter e
Starting at letter f
Starting at letter g
Starting at letter h
Starting at letter i
Starting at letter j
Starting at letter k
Starting at letter l
Starting at letter m
Starting at letter n
Starting at letter o
Starting at letter p
Starting at letter q
Starting at letter r
Starting at letter s
Starting at letter t
Starting at letter u
Starting at letter v
Starting at letter w
Starting at letter x
Starting at letter y
Starting at letter z


In [15]:
'''
Validate results
'''

print(len(draftedBy))
pprint(draftedBy[:20])

9897
[{'draftedBy': 'Dallas Cowboys',
  'draftedOverall': '179',
  'draftedPos': 'RB',
  'draftedYear': '1970',
  'playerId': 'don-abbey-1',
  'round': '7'},
 {'draftedBy': 'Green Bay Packers',
  'draftedOverall': '176',
  'draftedPos': 'WR',
  'draftedYear': '2014',
  'playerId': 'jared-abbrederis-1',
  'round': '5'},
 {'draftedBy': 'Miami Dolphins',
  'draftedOverall': '80',
  'draftedPos': 'RB',
  'draftedYear': '1996',
  'playerId': 'karim-abdul-jabbar-1',
  'round': '3'},
 {'draftedBy': 'Detroit Lions',
  'draftedOverall': '54',
  'draftedPos': 'RB',
  'draftedYear': '2015',
  'playerId': 'ameer-abdullah-1',
  'round': '2'},
 {'draftedBy': 'Tampa Bay Buccaneers',
  'draftedOverall': '231',
  'draftedPos': 'DB',
  'draftedYear': '2005',
  'playerId': 'hamza-abdullah-1',
  'round': '7'},
 {'draftedBy': 'Kansas City Chiefs',
  'draftedOverall': '224',
  'draftedPos': 'WR',
  'draftedYear': '1988',
  'playerId': 'azizuddin-abdur-raoof-1',
  'round': '9'},
 {'draftedBy': 'Detroit Lions

In [211]:
'''
Save the draftedBy object as a json file.  Needed for matching logic.
'''

with open("draftedBy.json", 'w') as outfile:
        json.dump(draftedBy, outfile)

In [17]:
'''
Manually create nested arrays preparing for conversion to pandas df.
'''

table = [["playerId", "draftedBy", "draftedOverall", "round", "draftedPos", "draftedYear"]]

for item in draftedBy:
    temp = []
    temp.append(item["playerId"])
    temp.append(item["draftedBy"])
    temp.append(item["draftedOverall"])
    temp.append(item["round"])
    temp.append(item["draftedPos"])
    temp.append(item["draftedYear"])
    table.append(temp)

In [19]:
'''
Validate results
'''

pprint(table[:20])

[['playerId',
  'draftedBy',
  'draftedOverall',
  'round',
  'draftedPos',
  'draftedYear'],
 ['don-abbey-1', 'Dallas Cowboys', '179', '7', 'RB', '1970'],
 ['jared-abbrederis-1', 'Green Bay Packers', '176', '5', 'WR', '2014'],
 ['karim-abdul-jabbar-1', 'Miami Dolphins', '80', '3', 'RB', '1996'],
 ['ameer-abdullah-1', 'Detroit Lions', '54', '2', 'RB', '2015'],
 ['hamza-abdullah-1', 'Tampa Bay Buccaneers', '231', '7', 'DB', '2005'],
 ['azizuddin-abdur-raoof-1', 'Kansas City Chiefs', '224', '9', 'WR', '1988'],
 ['david-abercrombie-1', 'Detroit Lions', '332', '13', 'QB/RB', '1971'],
 ['walter-abercrombie-1', 'Pittsburgh Steelers', '12', '1', 'RB', '1982'],
 ['dirk-abernathy-1', 'Oakland Raiders', '316', '12', 'DB', '1979'],
 ['victor-abiamiri-1', 'Philadelphia Eagles', '57', '2', 'DL', '2007'],
 ['derek-abney-1', 'Baltimore Ravens', '244', '7', 'WR', '2004'],
 ['oday-aboushi-1', 'New York Jets', '141', '5', 'OL', '2013'],
 ['clifton-abraham-1', 'Tampa Bay Buccaneers', '143', '5', 'DB', '1

## Append Matching professional drafted by position

In [20]:
'''
Load the matches file to a dictionary.
'''

with open('matches.json') as json_file:
    matches = json.load(json_file)

In [21]:
'''
Create nessted array of college id and matching nfl position preparing for conversion to pandas df.
'''

table_matches = [["college_id", "nfl_position"]]

for item in matches:
    temp = []
    temp.append(item["college_id"])
    temp.append(item["nfl_position"])
    table_matches.append(temp)

In [22]:
'''
Create pandas df of the draftedBy table and the matched nfl position tables.
'''

dfMatches = pd.DataFrame(table_matches[1:], columns = table_matches[0])
df = pd.DataFrame(table[1:], columns = table[0])

In [23]:
'''
Join the draftedBy table and the matched nfl position tables.
'''

join = pd.merge(df, dfMatches, 
                how='left', 
                left_on = 'playerId', 
                right_on = 'college_id').drop("college_id",1)

In [24]:
'''
Validate results.
'''

join.head()

,playerId,draftedBy,draftedOverall,round,draftedPos,draftedYear,nfl_position
0,don-abbey-1,Dallas Cowboys,179,7,RB,1970,NaN
1,jared-abbrederis-1,Green Bay Packers,176,5,WR,2014,WR
2,karim-abdul-jabbar-1,Miami Dolphins,80,3,RB,1996,NaN
3,ameer-abdullah-1,Detroit Lions,54,2,RB,2015,RB
4,hamza-abdullah-1,Tampa Bay Buccaneers,231,7,DB,2005,DB


In [26]:
'''
Save the joined table to a csv for the final draftBy table.
'''

join.to_csv("draftedBy.csv",index=False)

## Concatenate skill positions 

In [27]:
'''
Load in seperate receiving and rushing csv files
'''

dfRush = pd.read_csv('all_rushing.csv')
dfRec = pd.read_csv('all_receiving.csv')

In [28]:
'''
Since these two tables have the same columns and column positions.  Use pandas concat to combine the.
'''

dfSkill = pd.concat([dfRush, dfRec])

In [29]:
'''
Validate results.
'''

dfSkill.head()

,playerId,year,school_name,conf_abbr,class,pos,g,rush_att,rush_yds,rush_yds_per_att,rush_td,rec,rec_yds,rec_yds_per_rec,rec_td,scrim_att,scrim_yds,scrim_yds_per_att,scrim_td
0,don-abbey-1,1967,Penn State,Ind,NaN,RB,10.0,93.0,386.0,4.2,5.0,10.0,148.0,14.8,4.0,103,534,5.2,9
1,don-abbey-1,1968,Penn State,Ind,NaN,RB,10.0,39.0,164.0,4.2,3.0,1.0,14.0,14.0,0.0,40,178,4.5,3
2,don-abbey-1,1969,Penn State,Ind,NaN,RB,10.0,42.0,228.0,5.4,4.0,2.0,3.0,1.5,0.0,44,231,5.3,4
3,karim-abdul-jabbar-1,1995,UCLA,Pac-10,NaN,RB,12.0,296.0,1571.0,5.3,12.0,20.0,188.0,9.4,1.0,316,1759,5.6,13
4,ameer-abdullah-1,2011,Nebraska,Big Ten,FR,RB,13.0,42.0,150.0,3.6,3.0,1.0,11.0,11.0,0.0,43,161,3.7,3


In [30]:
'''
Check columns in new table.
'''

dfSkill.columns

Index(['playerId', 'year', 'school_name', 'conf_abbr', 'class', 'pos', 'g',
       'rush_att', 'rush_yds', 'rush_yds_per_att', 'rush_td', 'rec', 'rec_yds',
       'rec_yds_per_rec', 'rec_td', 'scrim_att', 'scrim_yds',
       'scrim_yds_per_att', 'scrim_td'],
      dtype='object')

In [31]:
'''
Save the dfskill object to csv.
'''

dfSkill.to_csv("offSkillPlayers.csv",index=False)